In [ ]:
import torch

# Sample random data for experiments

In [ ]:
n_dim=4
e_dim=4
eh_heads = 2
nh_heads = 3
n_nodes = 5
n_edges_sample = 15

In [ ]:
nodes = torch.randn(n_nodes, n_dim)

edges = torch.randint(low=0, high=n_nodes, size=(2, n_edges_sample))

order = torch.sort(edges[1]).indices
edges=edges[:, order]
order = torch.sort(edges[0]).indices
edges=edges[:, order]
edges = torch.unique(edges, dim=1)
edges = edges[:, edges[0, :] != edges[1, :]]

n_edges = edges.shape[1]

edge_features = torch.randn(n_edges, e_dim)

In [ ]:
edges

tensor([[1, 1, 2, 2, 3, 3, 3, 4],
        [2, 4, 0, 3, 1, 2, 4, 0]])

In [ ]:
edge_features

tensor([[ 0.2686,  0.1457, -0.6484, -2.2825],
        [-0.0046,  0.9504, -0.1812,  0.8982],
        [ 0.8177, -0.0413, -0.0633, -0.9602],
        [-0.4971,  0.3030,  0.9113, -0.4091],
        [ 1.3522, -0.4655, -0.8157,  1.1900],
        [ 2.4739,  0.1505,  1.6291, -0.0673],
        [ 0.9679,  3.1103,  0.7519,  2.5047],
        [ 0.6992,  1.4276, -0.1395,  1.1929]])

In [ ]:
nodes

tensor([[-0.7369,  0.0880, -0.0748, -0.0388],
        [ 0.6947, -0.7285,  1.3022,  0.7977],
        [ 0.9511, -0.3073, -0.3575,  0.8520],
        [-0.1284,  0.8181,  0.4489,  0.2868],
        [-0.7065,  1.3118, -0.0299,  1.5650]])

In [ ]:
mask = torch.zeros((5, 5))
mask[edges[0], edges[1]] = 1
mask

tensor([[0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 1.],
        [1., 0., 0., 1., 0.],
        [0., 1., 1., 0., 1.],
        [1., 0., 0., 0., 0.]])

# Experiments

## define layers

In [ ]:
import torch.nn as nn

# n_dim=4
# e_dim=3
# eh_heads = 2
# nh_heads = 3

In [ ]:
W_nv = nn.Linear(n_dim, n_dim * nh_heads)
W_nq = nn.Linear(n_dim, n_dim * nh_heads)
W_nk = nn.Linear(n_dim, n_dim * nh_heads)

In [ ]:
W_ev = nn.Linear(nh_heads, e_dim * eh_heads)
W_eq = nn.Linear(e_dim, e_dim * eh_heads)
W_ek = nn.Linear(nh_heads, e_dim * eh_heads)

In [ ]:
W_nr = nn.Linear(n_dim * nh_heads, n_dim)
W_er = nn.Linear(e_dim * eh_heads, nh_heads)

## nodes ->

In [ ]:
nv = W_nv(nodes)
nq = W_nq(nodes)
nk = W_nk(nodes)

In [ ]:
nv.shape

torch.Size([5, 12])

In [ ]:
nv

tensor([[ 0.7721,  0.3507, -0.5177, -0.5645, -0.7706,  0.2676,  0.0755, -0.5668,
          0.0247, -0.3495, -0.1441,  0.3607],
        [ 0.1298, -0.6281,  0.8674,  0.4509,  0.8572, -0.0534, -0.8120,  0.3037,
          0.1307, -0.0501,  0.2981, -0.2879],
        [ 0.5411, -0.4341,  0.2171,  0.2226,  0.3863, -0.1235, -0.0896, -0.1024,
         -0.3692, -0.4643,  0.7433,  0.1519],
        [ 0.1134,  0.2308,  0.1086,  0.0849, -0.6353,  0.3501, -0.1980,  0.0881,
          0.1711, -0.1332, -0.3892, -0.2093],
        [ 0.7340,  0.5271,  0.1615,  0.4457, -0.8241,  0.8604,  0.3114, -0.0363,
          0.4225, -0.9980, -0.7338, -0.2779]], grad_fn=<AddmmBackward0>)

In [ ]:
nv = nv.view(-1, nh_heads, n_dim).transpose(0, 1)
nq = nq.view(-1, nh_heads, n_dim).transpose(0, 1)
nk = nk.view(-1, nh_heads, n_dim).transpose(0, 1)

In [ ]:
nv.shape

torch.Size([3, 5, 4])

In [ ]:
nv

tensor([[[ 0.7721,  0.3507, -0.5177, -0.5645],
         [ 0.1298, -0.6281,  0.8674,  0.4509],
         [ 0.5411, -0.4341,  0.2171,  0.2226],
         [ 0.1134,  0.2308,  0.1086,  0.0849],
         [ 0.7340,  0.5271,  0.1615,  0.4457]],

        [[-0.7706,  0.2676,  0.0755, -0.5668],
         [ 0.8572, -0.0534, -0.8120,  0.3037],
         [ 0.3863, -0.1235, -0.0896, -0.1024],
         [-0.6353,  0.3501, -0.1980,  0.0881],
         [-0.8241,  0.8604,  0.3114, -0.0363]],

        [[ 0.0247, -0.3495, -0.1441,  0.3607],
         [ 0.1307, -0.0501,  0.2981, -0.2879],
         [-0.3692, -0.4643,  0.7433,  0.1519],
         [ 0.1711, -0.1332, -0.3892, -0.2093],
         [ 0.4225, -0.9980, -0.7338, -0.2779]]], grad_fn=<TransposeBackward0>)

In [ ]:
# (B, n_heads, T, hidden_size) * (B, n_heads, hidden_size, T) = (B, n_heads, T2, T1)
scaled_qk_dense = torch.matmul(nq, nk.transpose(-2, -1)) / torch.sqrt(
  torch.tensor(nk.size(-1))
  )
scaled_qk_dense = scaled_qk_dense * mask
scaled_qk_dense.shape

torch.Size([3, 5, 5])

In [ ]:
scaled_qk_dense

tensor([[[ 0.0000,  0.0000, -0.0000,  0.0000,  0.0000],
         [-0.0000, -0.0000, -0.0029, -0.0000, -0.0128],
         [ 0.1515, -0.0000, -0.0000,  0.2007,  0.0000],
         [ 0.0000, -0.2141, -0.2076,  0.0000,  0.0267],
         [ 0.2698, -0.0000, -0.0000,  0.0000,  0.0000]],

        [[ 0.0000, -0.0000, -0.0000,  0.0000,  0.0000],
         [-0.0000, -0.0000, -0.4182, -0.0000, -0.2533],
         [-0.1801, -0.0000, -0.0000, -0.2617, -0.0000],
         [ 0.0000, -0.0087, -0.0696,  0.0000,  0.1392],
         [ 0.0490, -0.0000, -0.0000,  0.0000, -0.0000]],

        [[ 0.0000,  0.0000, -0.0000,  0.0000, -0.0000],
         [-0.0000,  0.0000,  0.4674,  0.0000,  0.2948],
         [-0.0424, -0.0000, -0.0000, -0.0500,  0.0000],
         [ 0.0000, -0.0091,  0.1058,  0.0000,  0.0832],
         [-0.0188, -0.0000,  0.0000, -0.0000,  0.0000]]],
       grad_fn=<MulBackward0>)

In [ ]:
torch.where(scaled_qk_dense[0,:,:]!=0)

(tensor([1, 1, 2, 2, 3, 3, 3, 4]), tensor([2, 4, 0, 3, 1, 2, 4, 0]))

In [ ]:
scaled_qk_dense[:, edges[0], edges[1]]

tensor([[-0.0029, -0.0128,  0.1515,  0.2007, -0.2141, -0.2076,  0.0267,  0.2698],
        [-0.4182, -0.2533, -0.1801, -0.2617, -0.0087, -0.0696,  0.1392,  0.0490],
        [ 0.4674,  0.2948, -0.0424, -0.0500, -0.0091,  0.1058,  0.0832, -0.0188]],
       grad_fn=<IndexBackward0>)

In [ ]:
scaled_qk_sparse = (nq[:, edges[0], :] * nk[:, edges[1], :]).sum(-1)/ torch.sqrt(
  torch.tensor(nk.size(-1))
  )
scaled_qk_sparse.shape

torch.Size([3, 8])

In [ ]:
scaled_qk_sparse

tensor([[-0.0029, -0.0128,  0.1515,  0.2007, -0.2141, -0.2076,  0.0267,  0.2698],
        [-0.4182, -0.2533, -0.1801, -0.2617, -0.0087, -0.0696,  0.1392,  0.0490],
        [ 0.4674,  0.2948, -0.0424, -0.0500, -0.0091,  0.1058,  0.0832, -0.0188]],
       grad_fn=<DivBackward0>)

## edges ->

In [ ]:
ev = W_ev(scaled_qk_sparse.transpose(0, 1))
eq = W_eq(edge_features)
ek = W_ek(scaled_qk_sparse.transpose(0, 1))

In [ ]:
ev = ev.view(-1, eh_heads, e_dim)
eq = eq.view(-1, eh_heads, e_dim)
ek = ek.view(-1, eh_heads, e_dim)

In [ ]:
ev.shape

torch.Size([8, 2, 4])

In [ ]:
ev_dense = torch.sparse_coo_tensor(edges, ev, (n_nodes, n_nodes, eh_heads, e_dim)).to_dense().permute((2, 0, 1, 3))

In [ ]:
eq = torch.sparse_coo_tensor(edges, eq, (n_nodes, n_nodes, eh_heads, e_dim)).to_dense()
ek = torch.sparse_coo_tensor(edges, ek, (n_nodes, n_nodes, eh_heads, e_dim)).to_dense()
ev = torch.sparse_coo_tensor(edges, ev, (n_nodes, n_nodes, eh_heads, e_dim)).to_dense()

In [ ]:
ev.shape

torch.Size([5, 5, 2, 4])

In [ ]:
ev = ev.permute((2, 0, 1, 3))
eq = eq.permute((2, 0, 1, 3))
ek = ek.permute((2, 0, 1, 3))

In [ ]:
scaled_eqk_dense = torch.matmul(eq , ek.transpose(-2, -1)) / torch.sqrt(
  torch.tensor(ek.size(-1))
  )


In [ ]:
ev = W_ev(scaled_qk_sparse.transpose(0, 1))
eq = W_eq(edge_features)
ek = W_ek(scaled_qk_sparse.transpose(0, 1))

In [ ]:
ev = ev.view(-1, eh_heads, e_dim)
eq = eq.view(-1, eh_heads, e_dim)
ek = ek.view(-1, eh_heads, e_dim)

In [ ]:
edges

tensor([[1, 1, 2, 2, 3, 3, 3, 4],
        [2, 4, 0, 3, 1, 2, 4, 0]])

In [ ]:
a, b = torch.unique(edges[0, :], return_counts=True)
cumsum = torch.cumsum(b, dim=0)
cumsum = torch.stack([torch.hstack([torch.tensor([0]), cumsum])[:-1], cumsum, b])
eik = torch.cat([torch.arange(s, e, dtype=torch.int64).repeat(r) for s, e, r in cumsum.T])
eiq = torch.cat([torch.arange(s, e, dtype=torch.int64).repeat((r, 1)).transpose(0, 1).reshape(r**2) for s, e, r in cumsum.T])

indices_to_reduce_from_n3_shape = torch.stack([torch.repeat_interleave(a, b**2),
             edges[1, eiq],
             edges[1, eik],
             ])

In [ ]:
indices_to_reduce_from_n3_shape[2]

tensor([2, 4, 2, 4, 0, 3, 0, 3, 1, 2, 4, 1, 2, 4, 1, 2, 4, 0])

In [ ]:
ev.shape

torch.Size([8, 2, 4])

In [ ]:
scaled_eqk_sparse = (eq[eiq,:,:] * ek[eik,:,:]).sum(-1)/torch.sqrt(torch.tensor(ek.size(-1)))

In [ ]:
indices_to_reduce_from_n3_shape

tensor([[1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4],
        [2, 2, 4, 4, 0, 0, 3, 3, 1, 1, 1, 2, 2, 2, 4, 4, 4, 0],
        [2, 4, 2, 4, 0, 3, 0, 3, 1, 2, 4, 1, 2, 4, 1, 2, 4, 0]])

In [ ]:
scaled_eqk_sparse

tensor([[-0.3580,  0.2582],
        [-0.3553,  0.1997],
        [ 0.2449, -0.1392],
        [ 0.2507, -0.0659],
        [-0.2069,  0.1218],
        [-0.2061,  0.1365],
        [ 0.0326,  0.1796],
        [ 0.0293,  0.2036],
        [-0.0787,  0.0509],
        [-0.0927,  0.0535],
        [-0.0809,  0.0680],
        [-0.1561,  0.0969],
        [-0.1826,  0.1100],
        [-0.1032,  0.0604],
        [ 0.9218,  0.0523],
        [ 0.8961, -0.0078],
        [ 0.7457,  0.0518],
        [ 0.2940,  0.0767]], grad_fn=<DivBackward0>)

In [ ]:
scaled_eqk_dense.shape

torch.Size([2, 5, 5, 5])

In [ ]:
scaled_eqk_dense

tensor([[[[ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000]],

         [[ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000, -0.3580,  0.0000, -0.3553],
          [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.2449,  0.0000,  0.2507]],

         [[-0.2069,  0.0000,  0.0000, -0.2061,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
          [ 0.0326,  0.0000,  0.0000,  0.0293,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000]],

         [[ 0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
          [ 0.0000, -0.0787, -0.0927,  0.0000, -0.0809],
          [ 0.0000, -0.15

In [ ]:
indices_to_reduce_from_n3_shape

tensor([[1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4],
        [2, 2, 4, 4, 0, 0, 3, 3, 1, 1, 1, 2, 2, 2, 4, 4, 4, 0],
        [2, 4, 2, 4, 0, 3, 0, 3, 1, 2, 4, 1, 2, 4, 1, 2, 4, 0]])

In [ ]:
from torch_geometric.utils import softmax, scatter

In [ ]:
scaled_eqk_sparse = softmax(
    src=scaled_eqk_sparse,
    index=eiq,
    num_nodes=n_edges,
)

In [ ]:
scaled_eqk_dense[scaled_eqk_dense==0] = float("-inf")
scaled_eqk_dense = torch.nn.functional.softmax(scaled_eqk_dense, dim=-1).nan_to_num(0)
scaled_eqk_dense

tensor([[[[0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000, 0.0000]],

         [[0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.4993, 0.0000, 0.5007],
          [0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.4985, 0.0000, 0.5015]],

         [[0.4998, 0.0000, 0.0000, 0.5002, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
          [0.5008, 0.0000, 0.0000, 0.4992, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000, 0.0000]],

         [[0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.3351, 0.3305, 0.0000, 0.3344],
          [0.0000, 0.3302, 0.3216, 0.0000, 0.3482],
          [0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
      

In [ ]:
scaled_eqk_dense.shape

torch.Size([2, 5, 5, 5])

## edge <-

In [ ]:
edges

tensor([[1, 1, 2, 2, 3, 3, 3, 4],
        [2, 4, 0, 3, 1, 2, 4, 0]])

In [ ]:
scaled_eqk_sparse

tensor([[0.4993, 0.5146],
        [0.5007, 0.4854],
        [0.4985, 0.4817],
        [0.5015, 0.5183],
        [0.4998, 0.4963],
        [0.5002, 0.5037],
        [0.5008, 0.4940],
        [0.4992, 0.5060],
        [0.3351, 0.3311],
        [0.3305, 0.3320],
        [0.3344, 0.3369],
        [0.3302, 0.3359],
        [0.3216, 0.3403],
        [0.3482, 0.3238],
        [0.3555, 0.3400],
        [0.3465, 0.3202],
        [0.2981, 0.3398],
        [1.0000, 1.0000]], grad_fn=<DivBackward0>)

In [ ]:
scaled_eqk_sparse.unsqueeze(-1).shape, ev[eik,:,:].shape

(torch.Size([18, 2, 1]), torch.Size([18, 2, 4]))

In [ ]:
scaled_eqkv_sparse = scaled_eqk_sparse.unsqueeze(-1) * ev[eik,:,:]

In [ ]:
scaled_eqkv_sparse.shape

torch.Size([18, 2, 4])

In [ ]:
indices_to_reduce_from_n3_shape

tensor([[1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4],
        [2, 2, 4, 4, 0, 0, 3, 3, 1, 1, 1, 2, 2, 2, 4, 4, 4, 0],
        [2, 4, 2, 4, 0, 3, 0, 3, 1, 2, 4, 1, 2, 4, 1, 2, 4, 0]])

In [ ]:
e_result_sparse = scatter(scaled_eqkv_sparse, eiq, 0, dim_size=n_edges, reduce='sum')

In [ ]:
e_result_sparse

tensor([[[ 0.6403, -0.2865, -0.2470,  0.1731],
         [ 0.5066,  0.3076, -0.5122, -0.4644]],

        [[ 0.6403, -0.2866, -0.2470,  0.1731],
         [ 0.5016,  0.3079, -0.5118, -0.4632]],

        [[ 0.3692, -0.4656, -0.1010,  0.1291],
         [ 0.3924,  0.5164, -0.4191, -0.4785]],

        [[ 0.3693, -0.4656, -0.1010,  0.1291],
         [ 0.3925,  0.5165, -0.4190, -0.4785]],

        [[ 0.6849, -0.4063, -0.0815,  0.1831],
         [ 0.1400,  0.2753, -0.4655, -0.3530]],

        [[ 0.6844, -0.4086, -0.0809,  0.1802],
         [ 0.1397,  0.2759, -0.4626, -0.3520]],

        [[ 0.6860, -0.4004, -0.0826,  0.1907],
         [ 0.1394,  0.2754, -0.4658, -0.3530]],

        [[ 0.4465, -0.6026, -0.0480, -0.0339],
         [ 0.3048,  0.3886, -0.6223, -0.4897]]], grad_fn=<ScatterAddBackward0>)

In [ ]:
e_result_sparse.to_dense().permute((2, 0, 1, 3))

tensor([[[[ 0.0000,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  0.0000]],

         [[ 0.0000,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  0.0000],
          [ 0.6403, -0.2865, -0.2470,  0.1731],
          [ 0.0000,  0.0000,  0.0000,  0.0000],
          [ 0.6403, -0.2866, -0.2470,  0.1731]],

         [[ 0.3692, -0.4656, -0.1010,  0.1291],
          [ 0.0000,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  0.0000],
          [ 0.3693, -0.4656, -0.1010,  0.1291],
          [ 0.0000,  0.0000,  0.0000,  0.0000]],

         [[ 0.0000,  0.0000,  0.0000,  0.0000],
          [ 0.6849, -0.4063, -0.0815,  0.1831],
          [ 0.6844, -0.4086, -0.0809,  0.1802],
          [ 0.0000,  0.0000,  0.0000,  0.0000],
          [ 0.6860, -0.4004, -0.0826,  0.1907]],

         [[ 0.4465, -0.6026, -0.

In [ ]:
scaled_eqk_dense.shape, ev_dense.shape

(torch.Size([2, 5, 5, 5]), torch.Size([2, 5, 5, 4]))

In [ ]:
e_result_dense = torch.matmul(scaled_eqk_dense, ev_dense)

In [ ]:
e_result_dense

tensor([[[[ 0.0000,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  0.0000]],

         [[ 0.0000,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  0.0000],
          [ 0.6403, -0.2865, -0.2470,  0.1731],
          [ 0.0000,  0.0000,  0.0000,  0.0000],
          [ 0.6403, -0.2866, -0.2470,  0.1731]],

         [[ 0.3692, -0.4656, -0.1010,  0.1291],
          [ 0.0000,  0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000,  0.0000],
          [ 0.3693, -0.4656, -0.1010,  0.1291],
          [ 0.0000,  0.0000,  0.0000,  0.0000]],

         [[ 0.0000,  0.0000,  0.0000,  0.0000],
          [ 0.6849, -0.4063, -0.0815,  0.1831],
          [ 0.6844, -0.4086, -0.0809,  0.1802],
          [ 0.0000,  0.0000,  0.0000,  0.0000],
          [ 0.6860, -0.4004, -0.0826,  0.1907]],

         [[ 0.4465, -0.6026, -0.

In [ ]:
e_result_sparse.values().shape

torch.Size([8, 2, 4])

In [ ]:
e_result_dense.shape

torch.Size([2, 5, 5, 4])

## nodes <-

In [ ]:
we_result_sparse = W_er(e_result_sparse.values().view(-1, eh_heads*e_dim))

In [ ]:
we_result_sparse.shape

torch.Size([8, 3])

In [ ]:
edges

tensor([[1, 1, 2, 2, 3, 3, 3, 4],
        [2, 4, 0, 3, 1, 2, 4, 0]])

In [ ]:
we_result_dense = W_er(e_result_dense.permute((1, 2, 0, 3)).reshape(n_nodes, n_nodes, eh_heads*e_dim))
we_result_dense.shape

torch.Size([5, 5, 3])

In [ ]:
we_result_dense = (we_result_dense.permute((2, 0, 1)) * mask)

In [ ]:
we_result_dense

tensor([[[-0.0000e+00, -0.0000e+00, -0.0000e+00, -0.0000e+00, -0.0000e+00],
         [-0.0000e+00, -0.0000e+00, -1.1768e-01, -0.0000e+00, -1.1724e-01],
         [ 6.4597e-06, -0.0000e+00, -0.0000e+00, -1.4298e-05, -0.0000e+00],
         [-0.0000e+00,  4.4529e-03,  4.6122e-03, -0.0000e+00,  4.0285e-03],
         [ 3.5280e-02, -0.0000e+00, -0.0000e+00, -0.0000e+00, -0.0000e+00]],

        [[-0.0000e+00, -0.0000e+00, -0.0000e+00, -0.0000e+00, -0.0000e+00],
         [-0.0000e+00, -0.0000e+00, -3.0065e-01, -0.0000e+00, -2.9955e-01],
         [-2.1691e-01, -0.0000e+00, -0.0000e+00, -2.1691e-01, -0.0000e+00],
         [-0.0000e+00, -2.0847e-01, -2.0714e-01, -0.0000e+00, -2.1022e-01],
         [-2.0434e-01, -0.0000e+00, -0.0000e+00, -0.0000e+00, -0.0000e+00]],

        [[-0.0000e+00, -0.0000e+00, -0.0000e+00, -0.0000e+00, -0.0000e+00],
         [-0.0000e+00, -0.0000e+00, -4.0412e-01, -0.0000e+00, -4.0503e-01],
         [-2.9759e-01, -0.0000e+00, -0.0000e+00, -2.9754e-01, -0.0000e+00],
        

In [ ]:
we_result_dense.shape

torch.Size([3, 5, 5])

In [ ]:
we_result_sparse

tensor([[-1.1768e-01, -3.0065e-01, -4.0412e-01],
        [-1.1724e-01, -2.9955e-01, -4.0503e-01],
        [ 6.4671e-06, -2.1691e-01, -2.9759e-01],
        [-1.4275e-05, -2.1691e-01, -2.9754e-01],
        [ 4.4528e-03, -2.0847e-01, -5.0721e-01],
        [ 4.6122e-03, -2.0714e-01, -5.0680e-01],
        [ 4.0285e-03, -2.1022e-01, -5.0823e-01],
        [ 3.5280e-02, -2.0434e-01, -3.5431e-01]], grad_fn=<AddmmBackward0>)

In [ ]:
we_result_dense[we_result_dense==0] = float("-inf")
we_result_dense = torch.nn.functional.softmax(we_result_dense, dim=-1).nan_to_num(0)
we_result_dense

tensor([[[0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.4999, 0.0000, 0.5001],
         [0.5000, 0.0000, 0.0000, 0.5000, 0.0000],
         [0.0000, 0.3334, 0.3334, 0.0000, 0.3332],
         [1.0000, 0.0000, 0.0000, 0.0000, 0.0000]],

        [[0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.4997, 0.0000, 0.5003],
         [0.5000, 0.0000, 0.0000, 0.5000, 0.0000],
         [0.0000, 0.3334, 0.3338, 0.0000, 0.3328],
         [1.0000, 0.0000, 0.0000, 0.0000, 0.0000]],

        [[0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.5002, 0.0000, 0.4998],
         [0.5000, 0.0000, 0.0000, 0.5000, 0.0000],
         [0.0000, 0.3334, 0.3335, 0.0000, 0.3331],
         [1.0000, 0.0000, 0.0000, 0.0000, 0.0000]]],
       grad_fn=<NanToNumBackward0>)

In [ ]:
we_result_sparse.shape

torch.Size([8, 3])

In [ ]:
(we_result_sparse.unsqueeze(-1) * nv.transpose(0, 1)[edges[1], :, :]).shape

torch.Size([8, 3, 4])

In [ ]:
we_result_sparse = W_er(e_result_sparse.view(-1, eh_heads*e_dim))

In [ ]:
we_result_sparse = softmax(
    src=we_result_sparse,
    index=edges[0],
    num_nodes=n_nodes,
)

In [ ]:
result = scatter(we_result_sparse.unsqueeze(-1) * nv.transpose(0, 1)[edges[1], :, :],
        edges[0], 0, dim_size=n_nodes, reduce='sum')

In [ ]:
result

tensor([[[ 0.0000,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  0.0000]],

        [[ 0.6376,  0.0466,  0.1893,  0.3342],
         [-0.2192,  0.3687,  0.1111, -0.0693],
         [ 0.0265, -0.7310,  0.0051, -0.0629]],

        [[ 0.4427,  0.2908, -0.2045, -0.2398],
         [-0.7029,  0.3089, -0.0612, -0.2394],
         [ 0.0979, -0.2413, -0.2667,  0.0757]],

        [[ 0.4683, -0.1785,  0.4154,  0.3731],
         [ 0.1405,  0.2273, -0.1969,  0.0550],
         [ 0.0612, -0.5039,  0.1029, -0.1379]],

        [[ 0.7721,  0.3507, -0.5177, -0.5645],
         [-0.7706,  0.2676,  0.0755, -0.5668],
         [ 0.0247, -0.3495, -0.1441,  0.3607]]], grad_fn=<ScatterAddBackward0>)

In [ ]:
torch.matmul(we_result_dense, nv).transpose(0, 1)

tensor([[[ 0.0000,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  0.0000],
         [ 0.0000,  0.0000,  0.0000,  0.0000]],

        [[ 0.6376,  0.0466,  0.1893,  0.3342],
         [-0.2192,  0.3687,  0.1111, -0.0693],
         [ 0.0265, -0.7310,  0.0051, -0.0629]],

        [[ 0.4427,  0.2908, -0.2045, -0.2398],
         [-0.7029,  0.3089, -0.0612, -0.2394],
         [ 0.0979, -0.2413, -0.2667,  0.0757]],

        [[ 0.4683, -0.1785,  0.4154,  0.3731],
         [ 0.1405,  0.2273, -0.1969,  0.0550],
         [ 0.0612, -0.5039,  0.1029, -0.1379]],

        [[ 0.7721,  0.3507, -0.5177, -0.5645],
         [-0.7706,  0.2676,  0.0755, -0.5668],
         [ 0.0247, -0.3495, -0.1441,  0.3607]]], grad_fn=<TransposeBackward0>)

In [ ]:
! pip install torch_geometric torch_scatter

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 4.0 MB/s eta 0:00:00


In [ ]:
pip install pyg_lib torch_scatter torch_sparse torch_cluster torch_spline_conv -f https://data.pyg.org/whl/torch-2.1.0+cpu.html

Looking in links: https://data.pyg.org/whl/torch-2.1.0+cpu.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 32.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 497.3/497.3 kB 25.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 32.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 745.7/745.7 kB 42.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 208.5/208.5 kB 19.2 MB/s eta 0:00:00


In [ ]:
from torch_geometric.utils import scatter
